In [20]:
# Carga de librerias
# ------------------
# Librerias de uso general
import holidays
from google.cloud import storage

# Manejo de datos
import numpy as np
import pandas as pd

import kagglehub
import shutil
import os

from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import pickle
import numpy as np
from sklearn.ensemble import IsolationForest

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Estadística y series temporales
from statsmodels.tsa.seasonal import seasonal_decompose

# Se importan las funciones
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [21]:
column_mapper = {
    'Service': 'fecha',
    'Relational Database Service($)': 'rds',
    'EC2-Instances($)': 'ec2',
    'FSx($)': 'fsx',
    'Elastic File System($)': 'efs',
    'EC2-Other($)': 'ec2_other',
    'CloudWatch($)': 'cloudwatch',
    'Elastic Load Balancing($)': 'elb',
    'S3($)': 's3',
    'Backup($)': 'backup',
    'Key Management Service($)': 'kms',
    'DataSync($)': 'data_sync',
    'Secrets Manager($)': 'secrets_manager',
    'Resilience Hub($)': 'resiliency',
    'Total costs($)': 'total_costs'
}

Carga de los Datos
Se hace la carga de los datos desde la ruta Data 

In [22]:
# Datos con corte a Abril usar: "costs.csv"

df = pd.read_csv('/Users/angeleduardogamarrarios/Repositorio_UDEM/MLops_AMICO/data/costs.csv')
df.head()

,Service,Relational Database Service($),EC2-Instances($),FSx($),Elastic File System($),EC2-Other($),CloudWatch($),S3($),Elastic Load Balancing($),Backup($),Key Management Service($),DataSync($),Secrets Manager($),Resilience Hub($),Total costs($)
0,Service total,34131.482733,23531.788153,5152.073356,2830.548352,2132.939335,1543.902136,778.604880,758.963353,530.170434,91.476443,34.06381,0.178930,0.0,71516.191916
1,2024-06-01,31.851525,4.612654,12.472440,1.789663,0.611443,0.036701,1.156349,2.160000,0.281071,0.234161,NaN,0.000255,0.0,55.206262
2,2024-06-02,61.039787,NaN,12.460864,1.789663,0.322181,0.016653,1.126354,2.160000,0.281071,0.235058,NaN,0.000295,NaN,79.431926
3,2024-06-03,71.416393,63.161250,12.654853,1.790544,5.613880,0.429317,1.119325,2.160042,0.281071,0.240077,NaN,0.000225,NaN,158.866978
4,2024-06-04,54.833184,68.402379,12.658942,1.791199,5.682819,0.459109,1.116106,2.160139,0.281314,0.238019,NaN,0.000225,NaN,147.623435


In [23]:
# Mapeo de columnas 
df.rename(columns=column_mapper, inplace=True)
# Elimina la fila con índice 0 y ka columna total_cost
df = df.drop(0, axis=0)
df = df.drop('total_costs', axis=1)

# Prepocesar datos

Para nuestro caso de analisis vamos a aplicar una normalizacion

In [24]:
#Separar las variables continuas de las demás
df_continuas=df[["rds",
                       "ec2",
                       "fsx",
                       "efs",
                       "ec2_other",
                       "cloudwatch",
                       "s3",
                       "elb",
                       "backup",
                       "kms",
                       "data_sync",
                       "secrets_manager",
                       "resiliency"]]

In [25]:
# EL conjunto de datos se divide en una proporción de 60% (Entrenamiento), 20% (Validación) y 20% (Prueba).
X_train_val, X_test = train_test_split(df_continuas, test_size=0.2, random_state=42)
X_train, X_val = train_test_split(X_train_val, test_size=0.25, random_state=42) # 0.25 * 0.80 = 0.20 (20% para Validation)

In [26]:
# Normalizar los datos 
preprocessor_pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor_pipeline.fit(X_train)

X_train_scaled = preprocessor_pipeline.transform(X_train)
X_val_scaled = preprocessor_pipeline.transform(X_val)
X_test_scaled = preprocessor_pipeline.transform(X_test)

# 4. Verificación y Persistencia (MLOps)
print(f"Media de 'rds' en X_train original: {X_train['rds'].mean():.4f}")
print(f"Media de 'rds' en X_train escalado: {X_train_scaled[:, X_train.columns.get_loc('rds')].mean():.4f}") # La media debe ser cercana a 0

# Guardar el Pipeline (¡Esto es clave para el MLOps!)
# Debes guardar el objeto preprocessor_pipeline para usarlo en producción
# cuando llegue un nuevo dato de vino.

OUTPUT_DIR = "./data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

with open(os.path.join(OUTPUT_DIR, "preprocessor_pipeline.pkl"), "wb") as f:
    pickle.dump(preprocessor_pipeline, f)

print("-" * 50)
print(f"✅ Pipeline y datos escalados listos. El preprocesador se guardó en: {OUTPUT_DIR}")


Media de 'rds' en X_train original: 96.0687
Media de 'rds' en X_train escalado: -0.0000
--------------------------------------------------
✅ Pipeline y datos escalados listos. El preprocesador se guardó en: ./data/


Isolation Forest (El Enfoque Robusto)
El Isolation Forest es rápido, escalable y muy robusto para datos multivariados con muchos outliers.

In [30]:
# --- CONFIGURACIÓN DEL MODELO ---

# El parámetro 'contamination' es crucial.
# Define la proporción esperada de anomalías (ej. 1% a 5% del total de días)
# Si no estás seguro, empieza con un valor conservador (0.01).
contamination_factor = 0.01

# Inicializar el modelo
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=contamination_factor,
    random_state=42,
    n_jobs=-1  # Usa todos los núcleos
)

# Entrenar el modelo (No supervisado: no necesita etiquetas)
iso_forest.fit(X_train)

# Predecir las anomalías: -1 para anomalía, 1 para dato normal
df_=X_train
df_['anomaly_iso_forest'] = iso_forest.predict(X_train)

# Obtener la 'Puntuación de Anomalía' (cuanto menor, más anómalo)
df_['score_iso_forest'] = iso_forest.decision_function(X_train)

# Mostrar los días anómalos
print("\n--- Anomalías detectadas por Isolation Forest (ejemplo) ---")
anomalies_if = df_[df_['anomaly_iso_forest'] == -1]
print(f"Total de anomalías detectadas: {len(anomalies_if)}")
print(anomalies_if.head())
# Nota: El umbral para la detección es ajustado por el 'contamination' que elegiste.

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- anomaly_iso_forest
